In [1]:
import pandas as pd
import numpy as np

In [2]:
V = ['S1', 'S2', 'S3', 'S4', 'S5', 'Moyenne', 'Eligible']

data = {
    "N_candidat": ["0001", "0002", "0003", "0004", "0005"],
    "S1":[15, 13, 16, 17, 13],
    "S2": [11, 9, 8, 12, 10],
    "S3":[14, 12, 11, 15, 13],
    "S4": [16, 15, 15, 18, 15],
    "S5": [11, 11, 11, 11, 11],

    "Logique": [1, 1, 1, 0, 0],
    "BDD": [1, 1, 1, 0, 0],
    "PS": [1, 1, 1, 1, 0],
    "Moyenne": [2, 0, 1, 2, 0],
    "Eligible": [True, True, True, False, False],
    "Decision": ['A','R','L','R','R']
}
df= pd.DataFrame(data)
df

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,11,14,16,11,1,1,1,2,True,A
1,0002,13,9,12,15,11,1,1,1,0,True,R
2,0003,16,8,11,15,11,1,1,1,1,True,L
3,0004,17,12,15,18,11,0,0,1,2,False,R
4,0005,13,10,13,15,11,0,0,0,0,False,R


# ACi
## AC1

In [3]:
#ligne existe, la décision est bonne et toutes les variables de X ont les bonnes valeurs aussi
# j'ai supp la partie; and all((str(c.loc[:, [a]].iloc[0, 0]) == str(X[a])) for a in X) pour optimiser le combinatoire
def AC1(N, X, decision, df=df):
    c = df[df.N_candidat == N].copy()
    return len(c) == 1 and  c.Decision.iloc[0] == decision and all((str(c.loc[:, [a]].iloc[0, 0]) == str(X[a])) for a in X) 


In [4]:
def test_AC1():
    # Cas de succès
    assert AC1("0001", {"Logique":"1"}, 'A')
    assert AC1("0001", {"S1":"15", "BDD":"1", "Moyenne":"2", 'Eligible': True}, 'A')
    assert AC1("0004", {"S2":"12", "BDD":"0"}, 'R')

    # Cas d'échec
    assert not AC1("xxxx", {"S1":"15", "BDD":"1", "Moyenne":"2", 'Eligible': True}, 'A')
    display("AC1 est bien correcte.")

test_AC1()

'AC1 est bien correcte.'

## AC2

In [5]:
coeffs= [1, 1, 1, 1, 1] #calcul de la Moyenne
seuils = [12, 12, 12] # Calcul du status d'éligibilité (logique, BDD, PS)
thresholds = [12, 14] # prendre la décision

def med(S1, S2, S3, S4, S5, Logique, BDD, PS, coeffs=coeffs, seuils=seuils, thresholds2=thresholds):
    moyenne = (S1 * coeffs[0] + S2 * coeffs[1] + S3 * coeffs[2] + S4 * coeffs[3] + S5 * coeffs[4]) / sum(coeffs)
    e = (Logique*BDD*PS==1)
    m = (moyenne>thresholds2[1])*2 + (thresholds2[0] <= moyenne <= thresholds2[1])
    if e and m==2:
        d= 'A'
    elif e and m==1:
        d='L'
    else:
        d='R'
    return m, e, d

In [8]:
# Cas de succès
print("Cas succès 1 : ", AC2("0001", {"S1": 0},'A'))
print("Cas succès 2 : ", AC2("0002", {"S1": 19, "S5": 14}, 'R'))
print("Cas succès 3 : ", AC2("0003", {"Logique": 0, "S5": 19, "S2":14},'L'))

# # Cas d'échec
print("Cas échec 1 : ", AC2("0004", {"S5": 13, "BDD": 1},'R'))
print("Cas échec 2 : ", AC2("0005", {"S1": 13, "BDD": 0}, 'R'))


Cas succès 1 :  True
Cas succès 2 :  True
Cas succès 3 :  True
Cas échec 1 :  False
Cas échec 2 :  False


## AC3:

In [18]:
from itertools import combinations

def AC3(N, X, decision ,df=df):
    keys = list(X.keys())
    subsets = [dict(zip(combo, (X[key] for key in combo))) for r in range(1, len(keys)) for combo in combinations(keys, r)]
    return all(not AC1(N, subset, decision, df) or not AC2(N, subset, decision, affichage=False) for subset in subsets)


In [20]:
display(df)

# Cas de succès
print("Cas de succès:", AC3("0001", {"Logique": 0}, 'A'))
print("Cas de succès:", AC3("0004", {"Logique": 1, "BDD": 1}, 'R'))

# Cas d'échec
print("Cas d'échec:", AC3("0001", {"Logique": 0, "PS": 0}, 'A'))
print("Cas d'échec:", AC3("0004", {"Logique": 1, "BDD": 1, "S1": 16}, 'R'))



,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,11,14,16,11,1,1,1,2,True,A
1,0002,13,9,12,15,11,1,1,1,0,True,R
2,0003,16,8,11,15,11,1,1,1,1,True,L
3,0004,17,12,15,18,11,0,0,1,2,False,R
4,0005,13,10,13,15,11,0,0,0,0,False,R


Cas de succès: True
Cas de succès: True
Cas d'échec: False
Cas d'échec: False


In [11]:
def AC(N, X, decision, df=df):
    return AC1(N, X, decision, df) and AC2(N, X, decision, df) and AC3(N, X, decision, df)

In [471]:
""" from itertools import combinations

def AC3(N, X_cause, X_new, decision ,df=df, Calcul_D=med):
    keys = list(X_cause.keys())
    subsets_X_cause= [dict(zip(combo, (X_cause[key] for key in combo))) for r in range(1, len(keys)+1) for combo in combinations(keys, r)]
    
    keys = list(X_new.keys())
    subsets_X_new= [dict(zip(combo, (X_new[key] for key in combo))) for r in range(1, len(keys)+1) for combo in combinations(keys, r)]
    
    return all(not AC1(N, subsets_X_cause, decision, df) and not AC2(N, subsets_X_new, df, Calcul_D) for subsets_X_cause,subsets_X_new in zip(subsets_X_cause,subsets_X_new))
 """

' from itertools import combinations\n\ndef AC3(N, X_cause, X_new, decision ,df=df, Calcul_D=med):\n    keys = list(X_cause.keys())\n    subsets_X_cause= [dict(zip(combo, (X_cause[key] for key in combo))) for r in range(1, len(keys)+1) for combo in combinations(keys, r)]\n    \n    keys = list(X_new.keys())\n    subsets_X_new= [dict(zip(combo, (X_new[key] for key in combo))) for r in range(1, len(keys)+1) for combo in combinations(keys, r)]\n    \n    return all(not AC1(N, subsets_X_cause, decision, df) and not AC2(N, subsets_X_new, df, Calcul_D) for subsets_X_cause,subsets_X_new in zip(subsets_X_cause,subsets_X_new))\n '

# BCi
## BC1 et BC2

In [472]:
def BC1(N,X, decision, df=df):
    for r in range(1, len(V) + 1):
        subsets = combinations(V, r)
        for subset in subsets:
            X_subset = {key: X[0][key] for key in subset if key in X[0]}
            if AC(N, X_subset, decision[0], df):
                return True
    return False



In [481]:
display(df)

# Cas de succès
print("Cas de succès:", BC1("0001", ({"Logique": 0},_), ('A',_)))
print("Cas de succès:", BC1("0001", ({"Logique": 0, "BDD": 0},_), ('A',_)))
print("Cas de succès:", BC1("0001", ({"Logique": 0, "BDD": 0,"PS": 0, "Moyenne":1},_), ('A',_)))


# Cas d'échec
print("Cas d'échec:", BC1("0004", ({"Logique": 1, "BDD": 1, "S1":16},_), ('R',_)))
print("Cas d'échec:", BC1("0004", ({"Logique": 1, "BDD": 1, "S1": 16},_), ('R',_)))



,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,11,14,16,11,1,1,1,2,True,A
1,0002,13,9,12,15,11,1,1,1,0,True,R
2,0003,16,8,11,15,11,1,1,1,1,True,L
3,0004,17,12,15,18,11,0,0,1,2,False,R
4,0005,13,10,13,15,11,0,0,0,0,False,R


Cas de succès: True
Cas de succès: True
Cas de succès: True
Cas d'échec: False
Cas d'échec: False
